## SQL vs Spark

Examples borrowed from:
    * https://github.com/sbartek/intro-to-pyspark
    * https://github.com/carloapp2/SparkPOT.git
    
See doc on: http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#

Data comes from 
<https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data>

In [2]:
## Check if spark session is defined
## Otherwise create one with
#from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("PySparkShell").getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 spark://10.46.251.64:7077 
 AppName 
 Databricks Shell

In [3]:
!wget https://raw.githubusercontent.com/JulienCojan/pyspark_kschool/master/data/competitive-data-science-predict-future-sales/sales_train.csv.gz -P data/competitive-data-science-predict-future-sales/

# Dataframe

In [5]:
cities_rdd = sc.parallelize([
    ("MAD", "Madrid", "ES", 40.4165, -3.70256),
    ("BCN", "Barcelona", "ES", 41.297078, 2.078464),
    ("PAR", "Paris", "FR", 48.85341, 2.3488),
    ("ROM", "Rome", "IT", 41.89193, 12.51133)])
cities_rdd.collect()

Out[15]: [('MAD', 'Madrid', 'ES', 40.4165, -3.70256),
 ('BCN', 'Barcelona', 'ES', 41.297078, 2.078464),
 ('PAR', 'Paris', 'FR', 48.85341, 2.3488),
 ('ROM', 'Rome', 'IT', 41.89193, 12.51133)]

In [6]:
cities_df = cities_rdd.toDF(["city_code","city_name","country_code","latitude","longitude"])
cities_df

Out[16]: DataFrame[city_code: string, city_name: string, country_code: string, latitude: double, longitude: double]

In [7]:
cities_df.show()

+---------+---------+------------+---------+---------+
city_code|city_name|country_code| latitude|longitude|
+---------+---------+------------+---------+---------+
 MAD| Madrid| ES| 40.4165| -3.70256|
 BCN|Barcelona| ES|41.297078| 2.078464|
 PAR| Paris| FR| 48.85341| 2.3488|
 ROM| Rome| IT| 41.89193| 12.51133|
+---------+---------+------------+---------+---------+

In [8]:
import pyspark.sql.functions as F

In [9]:
cities_df.select(cities_df.city_code,F.col('city_name'),'latitude').show()

+---------+---------+---------+
city_code|city_name| latitude|
+---------+---------+---------+
 MAD| Madrid| 40.4165|
 BCN|Barcelona|41.297078|
 PAR| Paris| 48.85341|
 ROM| Rome| 41.89193|
+---------+---------+---------+

In [10]:
cities_df.select(F.lower(cities_df.city_code),F.col('city_name'))

Out[21]: DataFrame[lower(city_code): string, city_name: string]

In [11]:
cities_df.select(F.lower(cities_df.city_code),F.col('city_name')).toPandas()

In [12]:
#sales_path="data/competitive-data-science-predict-future-sales/sales_train.csv.gz"
sales_path="/FileStore/tables/sales_train_csv.gz"

In [13]:
sales_sdf = spark\
    .read\
    .option("header", "true")\
    .option("sep", ",")\
    .csv(sales_path)

In [14]:
sales_sdf.printSchema()

root
-- date: string (nullable = true)
-- date_block_num: string (nullable = true)
-- shop_id: string (nullable = true)
-- item_id: string (nullable = true)
-- item_price: string (nullable = true)
-- item_cnt_day: string (nullable = true)

In [15]:
sales_sdf.show(30,truncate=False)

+----------+--------------+-------+-------+----------+------------+
date |date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
02.01.2013|0 |59 |22154 |999.0 |1.0 |
03.01.2013|0 |25 |2552 |899.0 |1.0 |
05.01.2013|0 |25 |2552 |899.0 |-1.0 |
06.01.2013|0 |25 |2554 |1709.05 |1.0 |
15.01.2013|0 |25 |2555 |1099.0 |1.0 |
10.01.2013|0 |25 |2564 |349.0 |1.0 |
02.01.2013|0 |25 |2565 |549.0 |1.0 |
04.01.2013|0 |25 |2572 |239.0 |1.0 |
11.01.2013|0 |25 |2572 |299.0 |1.0 |
03.01.2013|0 |25 |2573 |299.0 |3.0 |
03.01.2013|0 |25 |2574 |399.0 |2.0 |
05.01.2013|0 |25 |2574 |399.0 |1.0 |
07.01.2013|0 |25 |2574 |399.0 |1.0 |
08.01.2013|0 |25 |2574 |399.0 |2.0 |
10.01.2013|0 |25 |2574 |399.0 |1.0 |
11.01.2013|0 |25 |2574 |399.0 |2.0 |
13.01.2013|0 |25 |2574 |399.0 |1.0 |
16.01.2013|0 |25 |2574 |399.0 |1.0 |
26.01.2013|0 |25 |2574 |399.0 |1.0 |
27.01.2013|0 |25 |2574 |399.0 |1.0 |
09.01.2013|0 |25 |2593 |279.0 |1.0 |
16.01.2013|0 |25 |2604 |299.0 |1.0 |
27.01.2013|0 |25 |2604 |299.0 |1.0 |
27.01.2013|0 |25 |2607 |279.0 |1.0 |
29.01.2013|0 |25 |2607 |279.0 |1.0 |
27.01.2013|0 |25 |2609 |1699.0 |1.0 |
06.01.2013|0 |25 |2548 |1708.95 |1.0 |
26.01.2013|0 |25 |2611 |299.0 |1.0 |
02.01.2013|0 |25 |2546 |299.0 |1.0 |
06.01.2013|0 |25 |2515 |1649.0 |1.0 |
+----------+--------------+-------+-------+----------+------------+
only showing top 30 rows

In [16]:
sales_sdf.count()

Out[41]: 2935849

In [17]:
v = sales_sdf.select(sales_sdf.date,sales_sdf.item_price).limit(20).collect()

In [18]:
v

Out[37]: [Row(date='02.01.2013', item_price='999.0'),
 Row(date='03.01.2013', item_price='899.0'),
 Row(date='05.01.2013', item_price='899.0'),
 Row(date='06.01.2013', item_price='1709.05'),
 Row(date='15.01.2013', item_price='1099.0'),
 Row(date='10.01.2013', item_price='349.0'),
 Row(date='02.01.2013', item_price='549.0'),
 Row(date='04.01.2013', item_price='239.0'),
 Row(date='11.01.2013', item_price='299.0'),
 Row(date='03.01.2013', item_price='299.0'),
 Row(date='03.01.2013', item_price='399.0'),
 Row(date='05.01.2013', item_price='399.0'),
 Row(date='07.01.2013', item_price='399.0'),
 Row(date='08.01.2013', item_price='399.0'),
 Row(date='10.01.2013', item_price='399.0'),
 Row(date='11.01.2013', item_price='399.0'),
 Row(date='13.01.2013', item_price='399.0'),
 Row(date='16.01.2013', item_price='399.0'),
 Row(date='26.01.2013', item_price='399.0'),
 Row(date='27.01.2013', item_price='399.0')]

In [19]:
pdf = sales_sdf.select(sales_sdf.date,sales_sdf.item_price).limit(20).toPandas()
pdf

,date,item_price
0,02.01.2013,999.0
1,03.01.2013,899.0
2,05.01.2013,899.0
3,06.01.2013,1709.05
4,15.01.2013,1099.0
5,10.01.2013,349.0
6,02.01.2013,549.0
7,04.01.2013,239.0
8,11.01.2013,299.0
9,03.01.2013,299.0


In [20]:
type(pdf)

Out[40]: pandas.core.frame.DataFrame

In [21]:
sales_sdf[['shop_id','item_id']].head()

Out[39]: Row(shop_id='59', item_id='22154')

In [22]:
sales_sdf.createOrReplaceTempView("sales")

In [23]:
sales_10 = spark.sql(
"""
SELECT *
FROM sales
LIMIT 10
""")
sales_10

Out[47]: DataFrame[date: string, date_block_num: string, shop_id: string, item_id: string, item_price: string, item_cnt_day: string]

In [24]:
sales_10.show()

+----------+--------------+-------+-------+----------+------------+
 date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
02.01.2013| 0| 59| 22154| 999.0| 1.0|
03.01.2013| 0| 25| 2552| 899.0| 1.0|
05.01.2013| 0| 25| 2552| 899.0| -1.0|
06.01.2013| 0| 25| 2554| 1709.05| 1.0|
15.01.2013| 0| 25| 2555| 1099.0| 1.0|
10.01.2013| 0| 25| 2564| 349.0| 1.0|
02.01.2013| 0| 25| 2565| 549.0| 1.0|
04.01.2013| 0| 25| 2572| 239.0| 1.0|
11.01.2013| 0| 25| 2572| 299.0| 1.0|
03.01.2013| 0| 25| 2573| 299.0| 3.0|
+----------+--------------+-------+-------+----------+------------+

In [25]:
sales_sdf.select(F.year(F.col('date'))).show()

+----------+
year(date)|
+----------+
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
 null|
+----------+
only showing top 20 rows

In [26]:
from pyspark.sql.types import DateType

In [27]:
sales_sdf2 = sales_sdf\
    .withColumnRenamed('date', 'date_str')\
    .withColumn('date', (F.from_unixtime(F.unix_timestamp(F.col('date_str'), 'dd.MM.yyyy'))).cast(DateType()))\
    .withColumn('date_str', F.from_unixtime(F.unix_timestamp(F.col('date_str'), 'dd.MM.yyyy')))
sales_sdf2.show()

+-------------------+--------------+-------+-------+----------+------------+----------+
 date_str|date_block_num|shop_id|item_id|item_price|item_cnt_day| date|
+-------------------+--------------+-------+-------+----------+------------+----------+
2013-01-02 00:00:00| 0| 59| 22154| 999.0| 1.0|2013-01-02|
2013-01-03 00:00:00| 0| 25| 2552| 899.0| 1.0|2013-01-03|
2013-01-05 00:00:00| 0| 25| 2552| 899.0| -1.0|2013-01-05|
2013-01-06 00:00:00| 0| 25| 2554| 1709.05| 1.0|2013-01-06|
2013-01-15 00:00:00| 0| 25| 2555| 1099.0| 1.0|2013-01-15|
2013-01-10 00:00:00| 0| 25| 2564| 349.0| 1.0|2013-01-10|
2013-01-02 00:00:00| 0| 25| 2565| 549.0| 1.0|2013-01-02|
2013-01-04 00:00:00| 0| 25| 2572| 239.0| 1.0|2013-01-04|
2013-01-11 00:00:00| 0| 25| 2572| 299.0| 1.0|2013-01-11|
2013-01-03 00:00:00| 0| 25| 2573| 299.0| 3.0|2013-01-03|
2013-01-03 00:00:00| 0| 25| 2574| 399.0| 2.0|2013-01-03|
2013-01-05 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-05|
2013-01-07 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-07|
2013-01-08 00:00:00| 0| 25| 2574| 399.0| 2.0|2013-01-08|
2013-01-10 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-10|
2013-01-11 00:00:00| 0| 25| 2574| 399.0| 2.0|2013-01-11|
2013-01-13 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-13|
2013-01-16 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-16|
2013-01-26 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-26|
2013-01-27 00:00:00| 0| 25| 2574| 399.0| 1.0|2013-01-27|
+-------------------+--------------+-------+-------+----------+------------+----------+
only showing top 20 rows

In [28]:
sales_sdf2.printSchema()

root
-- date_str: string (nullable = true)
-- date_block_num: string (nullable = true)
-- shop_id: string (nullable = true)
-- item_id: string (nullable = true)
-- item_price: string (nullable = true)
-- item_cnt_day: string (nullable = true)
-- date: date (nullable = true)

In [29]:
sales_sdf2.select(F.year(F.col('date_str'))).show()

+--------------+
year(date_str)|
+--------------+
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
 2013|
+--------------+
only showing top 20 rows

In [30]:
sales_sdf2.createOrReplaceTempView("sales2")

## SELECT ~ select

In [32]:
spark.sql("""
SELECT shop_id, item_id
FROM sales
""").show()

In [33]:
sales_sdf\
   .select("shop_id", "item_id")\
   .head()

In [34]:
spark.sql("""
SELECT DISTINCT shop_id
FROM sales
""").show()

In [35]:
spark.sql("""
SELECT DISTINCT shop_id, item_id
FROM sales
""").show()

+-------+-------+
shop_id|item_id|
+-------+-------+
 25| 2462|
 25| 5274|
 25| 4910|
 25| 5592|
 24| 5848|
 23| 21336|
 24| 3158|
 25| 13903|
 25| 16890|
 25| 16644|
 25| 16562|
 25| 16457|
 25| 15701|
 25| 8034|
 25| 8831|
 25| 7276|
 25| 11236|
 25| 12133|
 19| 15100|
 19| 14939|
+-------+-------+
only showing top 20 rows

In [36]:
sales_sdf\
    .select("shop_id","item_id")\
    .distinct()\
    .show()

__Exercise__ List distinct values for `date` and `date_block_num`.

In [38]:
sales_sdf\
    .select("date","date_block_num")\
    .distinct()\
    .show()

## WHERE ~ filter

In [40]:
spark.sql("""
SELECT *
FROM sales
WHERE shop_id = 25
""").show(truncate=False)

In [41]:
sales_sdf\
   .filter(F.col("shop_id") == 25)\
   .show()

+----------+--------------+-------+-------+----------+------------+
 date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
03.01.2013| 0| 25| 2552| 899.0| 1.0|
05.01.2013| 0| 25| 2552| 899.0| -1.0|
06.01.2013| 0| 25| 2554| 1709.05| 1.0|
15.01.2013| 0| 25| 2555| 1099.0| 1.0|
10.01.2013| 0| 25| 2564| 349.0| 1.0|
02.01.2013| 0| 25| 2565| 549.0| 1.0|
04.01.2013| 0| 25| 2572| 239.0| 1.0|
11.01.2013| 0| 25| 2572| 299.0| 1.0|
03.01.2013| 0| 25| 2573| 299.0| 3.0|
03.01.2013| 0| 25| 2574| 399.0| 2.0|
05.01.2013| 0| 25| 2574| 399.0| 1.0|
07.01.2013| 0| 25| 2574| 399.0| 1.0|
08.01.2013| 0| 25| 2574| 399.0| 2.0|
10.01.2013| 0| 25| 2574| 399.0| 1.0|
11.01.2013| 0| 25| 2574| 399.0| 2.0|
13.01.2013| 0| 25| 2574| 399.0| 1.0|
16.01.2013| 0| 25| 2574| 399.0| 1.0|
26.01.2013| 0| 25| 2574| 399.0| 1.0|
27.01.2013| 0| 25| 2574| 399.0| 1.0|
09.01.2013| 0| 25| 2593| 279.0| 1.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 20 rows

__Exercise__ List different items from shop 25.

In [43]:
spark.sql("""
SELECT DISTINCT item_id
FROM sales
WHERE shop_id = 25
""").show(truncate=False)

In [44]:
tmp_df = sales_sdf[(sales_sdf["shop_id"] == 25)]
tmp_df.show()

+----------+--------------+-------+-------+----------+------------+
 date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
03.01.2013| 0| 25| 2552| 899.0| 1.0|
05.01.2013| 0| 25| 2552| 899.0| -1.0|
06.01.2013| 0| 25| 2554| 1709.05| 1.0|
15.01.2013| 0| 25| 2555| 1099.0| 1.0|
10.01.2013| 0| 25| 2564| 349.0| 1.0|
02.01.2013| 0| 25| 2565| 549.0| 1.0|
04.01.2013| 0| 25| 2572| 239.0| 1.0|
11.01.2013| 0| 25| 2572| 299.0| 1.0|
03.01.2013| 0| 25| 2573| 299.0| 3.0|
03.01.2013| 0| 25| 2574| 399.0| 2.0|
05.01.2013| 0| 25| 2574| 399.0| 1.0|
07.01.2013| 0| 25| 2574| 399.0| 1.0|
08.01.2013| 0| 25| 2574| 399.0| 2.0|
10.01.2013| 0| 25| 2574| 399.0| 1.0|
11.01.2013| 0| 25| 2574| 399.0| 2.0|
13.01.2013| 0| 25| 2574| 399.0| 1.0|
16.01.2013| 0| 25| 2574| 399.0| 1.0|
26.01.2013| 0| 25| 2574| 399.0| 1.0|
27.01.2013| 0| 25| 2574| 399.0| 1.0|
09.01.2013| 0| 25| 2593| 279.0| 1.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 20 rows

In [45]:
tmp_df = sales_sdf\
   .filter(sales_sdf["shop_id"] == 25)\
   .select('item_id')\
   .distinct()
tmp_df.show()
tmp_df.explain()

+-------+
item_id|
+-------+
 2088|
 3210|
 829|
 14899|
 13610|
 17506|
 6613|
 10096|
 11332|
 20158|
 18130|
 18947|
 6194|
 18634|
 19338|
 21331|
 15555|
 17401|
 6240|
 3959|
+-------+
only showing top 20 rows

== Physical Plan ==
*(2) HashAggregate(keys=[item_id#1715], functions=[])
+- Exchange hashpartitioning(item_id#1715, 200), [id=#5652]
 +- *(1) HashAggregate(keys=[item_id#1715], functions=[])
 +- *(1) Project [item_id#1715]
 +- *(1) Filter (isnotnull(shop_id#1714) && (cast(shop_id#1714 as int) = 25))
 +- *(1) FileScan csv [shop_id#1714,item_id#1715] Batched: false, DataFilters: [isnotnull(shop_id#1714), (cast(shop_id#1714 as int) = 25)], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/sales_train_csv.gz], PartitionFilters: [], PushedFilters: [IsNotNull(shop_id)], ReadSchema: struct<shop_id:string,item_id:string>

In [46]:
tmp_df2 = sales_sdf\
    .select(F.col('item_id'))\
    .distinct()\
    .filter(F.col("shop_id") == 25)
tmp_df2.show()
tmp_df2.explain()

+-------+
item_id|
+-------+
 10096|
 11332|
 11563|
 13610|
 14899|
 16576|
 17506|
 20428|
 20512|
 2088|
 2136|
 3210|
 6613|
 829|
 10272|
 10309|
 10603|
 11236|
 12542|
 14218|
+-------+
only showing top 20 rows

== Physical Plan ==
*(2) Project [item_id#1715]
+- *(2) Filter (isnotnull(shop_id#1714) && (cast(shop_id#1714 as int) = 25))
 +- SortAggregate(key=[item_id#1715], functions=[finalmerge_first(merge first#4790, valueSet#4791) AS first(shop_id#1714)()#4787])
 +- Sort [item_id#1715 ASC NULLS FIRST], false, 0
 +- Exchange hashpartitioning(item_id#1715, 200), [id=#6058]
 +- SortAggregate(key=[item_id#1715], functions=[partial_first(shop_id#1714, false) AS (first#4790, valueSet#4791)])
 +- *(1) Sort [item_id#1715 ASC NULLS FIRST], false, 0
 +- *(1) Project [item_id#1715, shop_id#1714]
 +- *(1) FileScan csv [shop_id#1714,item_id#1715] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[dbfs:/FileStore/tables/sales_train_csv.gz], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<shop_id:string,item_id:string>

In [47]:
tmp_df2.show()

## ORDER BY ~ orderBy

In [49]:
spark.sql("""
SELECT *
FROM sales
WHERE shop_id = 25 AND item_id = 2252
ORDER BY date desc
""").show()

In [50]:
sales_sdf\
   .filter((F.col("shop_id") == 25) & (F.col("item_id") == 2252))\
   .orderBy("date")\
   .show()

In [51]:
sales_sdf\
   .filter((F.col("shop_id") == 25) & (F.col("item_id") == 2252))\
   .orderBy(F.desc("item_cnt_day"))\
   .show()

__Exercise__ List different items and its price that were sold on 20th or 21st of August 2015 ordered by price starting from the most expensive.

In [53]:
sales_sdf\
   .filter((F.col("date") == "21.08.2015") | (F.col("date") == "22.08.2015"))\
   .orderBy(F.desc("item_price"))\
   .show()

In [54]:
sales_sdf\
   .filter(F.col("date").isin(["21.08.2015","22.08.2015"]))\
   .orderBy(F.desc("item_price"))\
   .show()

In [55]:
sales_sdf2\
   .filter(F.col("date").isin(["2015-08-21","2015-08-22"]))\
   .orderBy(F.desc("item_price"))\
   .show()

In [56]:
sales_sdf2\
   .filter((F.year(F.col("date"))==2015) & (F.month(F.col("date"))==8))\
   .orderBy(F.desc("item_price"))\
   .show()

In [57]:
spark.sql(
"""
select *
from sales2
where year(date)=2015 and month(date)=8
"""
).show()

## AS ~ alias

In [59]:
spark.sql("""
SELECT item_cnt_day
,   item_price
,   item_cnt_day * item_price AS revenue
FROM sales
""").show()

In [60]:
sales_sdf\
    .select(
        F.col("item_cnt_day"),
        F.col("item_price"),
        (F.col("item_cnt_day") * F.col("item_price")).alias("revenue")
    ).show()

+------------+----------+-------+
item_cnt_day|item_price|revenue|
+------------+----------+-------+
 1.0| 999.0| 999.0|
 1.0| 899.0| 899.0|
 -1.0| 899.0| -899.0|
 1.0| 1709.05|1709.05|
 1.0| 1099.0| 1099.0|
 1.0| 349.0| 349.0|
 1.0| 549.0| 549.0|
 1.0| 239.0| 239.0|
 1.0| 299.0| 299.0|
 3.0| 299.0| 897.0|
 2.0| 399.0| 798.0|
 1.0| 399.0| 399.0|
 1.0| 399.0| 399.0|
 2.0| 399.0| 798.0|
 1.0| 399.0| 399.0|
 2.0| 399.0| 798.0|
 1.0| 399.0| 399.0|
 1.0| 399.0| 399.0|
 1.0| 399.0| 399.0|
 1.0| 399.0| 399.0|
+------------+----------+-------+
only showing top 20 rows

## aggregators

In [62]:
spark.sql("""
SELECT AVG(item_cnt_day) AS mean_sale
,   STDDEV(item_cnt_day) AS sd_sales
,   SUM(item_cnt_day) AS sum_sales
,   COUNT(*) AS nitems
FROM sales
""").show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o208.sql.
: org.apache.spark.sql.AnalysisException: grouping expressions sequence is empty, and 'sales.`shop_id`' is not an aggregate function. Wrap '(avg(CAST(sales.`item_cnt_day` AS DOUBLE)) AS `mean_sale`, stddev_samp(CAST(sales.`item_cnt_day` AS DOUBLE)) AS `sd_sales`, sum(CAST(sales.`item_cnt_day` AS DOUBLE)) AS `sum_sales`, count(1) AS `nitems`)' in windowing function(s) or wrap 'sales.`shop_id`' in first() (or first_value) if you don't care which value you get.;;
Aggregate [avg(cast(item_cnt_day#1717 as double)) AS mean_sale#7755, stddev_samp(cast(item_cnt_day#1717 as double)) AS sd_sales#7756, sum(cast(item_cnt_day#1717 as double)) AS sum_sales#7757, count(1) AS nitems#7758L, shop_id#1714]
+- SubqueryAlias `sales`
 +- Relation[date#1712,date_block_num#1713,shop_id#1714,item_id#1715,item_price#1716,item_cnt_day#1717] csv

	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.failAnalysis(CheckAnalysis.scala:46)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.org$apache$spark$sql$catalyst$analysis$CheckAnalysis$class$$anonfun$$checkValidAggregateExpression$1(CheckAnalysis.scala:227)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$10.apply(CheckAnalysis.scala:268)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$10.apply(CheckAnalysis.scala:268)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:268)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:147)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.checkAnalysis(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:117)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:205)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:85)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:86)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$executePhase$1.apply(QueryExecution.scala:229)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:835)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:228)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:83)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:83)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:75)
	at org.apache.spark.sql.Dataset$$anonfun$ofRows$2.apply(Dataset.scala:90)
	at org.apache.spark.sql.Dataset$$anonfun$ofRows$2.

In [63]:
sales_sdf\
    .select(
        F.mean(F.col("item_cnt_day")).alias("mean_sales"),
        F.stddev(F.col("item_cnt_day")).alias("sd_sales"),
        F.sum(F.col("item_cnt_day")).alias("sum_sales"),
        F.count(F.col("item_cnt_day")).alias("n_items")
    ).show()

+-----------------+------------------+---------+-------+
 mean_sales| sd_sales|sum_sales|n_items|
+-----------------+------------------+---------+-------+
1.242640885140891|2.6188344308954035|3648206.0|2935849|
+-----------------+------------------+---------+-------+

__Exercise__ What is mean, standard deviation and median of the number of sold items?

In [65]:
spark.sql("""
SELECT AVG(item_cnt_day) AS mean_sale
,   STDDEV(item_cnt_day) AS sd_sales
,   SUM(item_cnt_day) AS sum_sales
,   COUNT(*) AS nitems
,   PERCENTILE(item_cnt_day, 0.5) as median
FROM sales
""").show()

+-----------------+------------------+---------+-------+------+
 mean_sale| sd_sales|sum_sales| nitems|median|
+-----------------+------------------+---------+-------+------+
1.242640885140891|2.6188344308954035|3648206.0|2935849| 1.0|
+-----------------+------------------+---------+-------+------+

In [66]:
sales_sdf.show(50)

+----------+--------------+-------+-------+----------+------------+
 date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
02.01.2013| 0| 59| 22154| 999.0| 1.0|
03.01.2013| 0| 25| 2552| 899.0| 1.0|
05.01.2013| 0| 25| 2552| 899.0| -1.0|
06.01.2013| 0| 25| 2554| 1709.05| 1.0|
15.01.2013| 0| 25| 2555| 1099.0| 1.0|
10.01.2013| 0| 25| 2564| 349.0| 1.0|
02.01.2013| 0| 25| 2565| 549.0| 1.0|
04.01.2013| 0| 25| 2572| 239.0| 1.0|
11.01.2013| 0| 25| 2572| 299.0| 1.0|
03.01.2013| 0| 25| 2573| 299.0| 3.0|
03.01.2013| 0| 25| 2574| 399.0| 2.0|
05.01.2013| 0| 25| 2574| 399.0| 1.0|
07.01.2013| 0| 25| 2574| 399.0| 1.0|
08.01.2013| 0| 25| 2574| 399.0| 2.0|
10.01.2013| 0| 25| 2574| 399.0| 1.0|
11.01.2013| 0| 25| 2574| 399.0| 2.0|
13.01.2013| 0| 25| 2574| 399.0| 1.0|
16.01.2013| 0| 25| 2574| 399.0| 1.0|
26.01.2013| 0| 25| 2574| 399.0| 1.0|
27.01.2013| 0| 25| 2574| 399.0| 1.0|
09.01.2013| 0| 25| 2593| 279.0| 1.0|
16.01.2013| 0| 25| 2604| 299.0| 1.0|
27.01.2013| 0| 25| 2604| 299.0| 1.0|
27.01.2013| 0| 25| 2607| 279.0| 1.0|
29.01.2013| 0| 25| 2607| 279.0| 1.0|
27.01.2013| 0| 25| 2609| 1699.0| 1.0|
06.01.2013| 0| 25| 2548| 1708.95| 1.0|
26.01.2013| 0| 25| 2611| 299.0| 1.0|
02.01.2013| 0| 25| 2546| 299.0| 1.0|
06.01.2013| 0| 25| 2515| 1649.0| 1.0|
07.01.2013| 0| 25| 2705| 1799.0| 1.0|
11.01.2013| 0| 25| 2705| 1799.0| 1.0|
26.01.2013| 0| 25| 2705| 1799.0| 1.0|
02.01.2013| 0| 25| 2715| 899.0| 1.0|
12.01.2013| 0| 25| 2715| 898.5| 1.0|
02.01.2013| 0| 25| 2716| 149.0| 1.0|
23.01.2013| 0| 25| 2716| 149.0| 1.0|
02.01.2013| 0| 25| 2719| 2699.0| 1.0|
12.01.2013| 0| 25| 2719| 2699.0| 1.0|
25.01.2013| 0| 25| 2719| 1699.0| 1.0|
30.01.2013| 0| 25| 2720| 1699.0| 1.0|
05.01.2013| 0| 25| 2473| 899.0| 1.0|
02.01.2013| 0| 25| 2480| 58.0| 1.0|
02.01.2013| 0| 25| 2731| 599.0| 1.0|
19.01.2013| 0| 25| 2731| 599.0| 1.0|
06.01.2013| 0| 25| 2746| 299.0| 1.0|
07.01.2013| 0| 25| 2746| 299.0| 1.0|
10.01.2013| 0| 25| 2746| 299.0| 2.0|
12.01.2013| 0| 25| 2746| 299.0| 1.0|
20.01.2013| 0| 25| 2746| 299.0| 1.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 50 rows

## GROUP BY

In [68]:
sales_sdf\
  .groupby("date", "shop_id")\
  .agg(
    F.sum(sales_sdf.item_cnt_day).alias("items_sold"),
    F.avg(sales_sdf.item_cnt_day).alias("avg_items_sold"))\
  .orderBy("shop_id","date")\
  .show()

+----------+-------+----------+------------------+
 date|shop_id|items_sold| avg_items_sold|
+----------+-------+----------+------------------+
01.02.2013| 0| 228.0|1.2127659574468086|
02.02.2013| 0| 317.0|1.3045267489711934|
03.01.2013| 0| 203.0|1.1666666666666667|
03.02.2013| 0| 200.0|1.2195121951219512|
04.01.2013| 0| 352.0|1.2941176470588236|
04.02.2013| 0| 121.0|1.1634615384615385|
05.01.2013| 0| 292.0|1.2372881355932204|
05.02.2013| 0| 173.0|1.1161290322580646|
06.01.2013| 0| 209.0|1.1807909604519775|
06.02.2013| 0| 194.0|1.1757575757575758|
07.01.2013| 0| 132.0|1.1379310344827587|
07.02.2013| 0| 135.0| 1.115702479338843|
08.01.2013| 0| 263.0|1.2464454976303319|
08.02.2013| 0| 287.0|1.2370689655172413|
09.01.2013| 0| 181.0| 1.13125|
09.02.2013| 0| 313.0|1.2620967741935485|
10.01.2013| 0| 184.0|1.1870967741935483|
10.02.2013| 0| 205.0|1.2349397590361446|
11.01.2013| 0| 139.0|1.1583333333333334|
11.02.2013| 0| 162.0| 1.2|
+----------+-------+----------+------------------+
only showing top 20 rows

In [69]:
spark.sql("""
SELECT 
    date_format(from_unixtime(unix_timestamp(date, 'dd.MM.yyyy')), "yyyy-MM-dd") AS date
,   SUM(item_cnt_day) AS items_sold
FROM sales
GROUP BY date
""").show()

+----------+----------+
 date|items_sold|
+----------+----------+
2013-02-16| 6643.0|
2014-02-09| 4646.0|
2014-09-01| 2887.0|
2014-10-18| 5001.0|
2015-06-27| 2563.0|
2015-09-17| 1887.0|
2013-04-29| 2771.0|
2013-04-12| 3947.0|
2014-09-18| 2441.0|
2015-08-15| 2201.0|
2015-10-28| 3593.0|
2013-02-05| 3302.0|
2013-09-21| 6698.0|
2014-05-31| 5395.0|
2014-11-02| 4390.0|
2015-07-08| 1905.0|
2015-09-13| 2660.0|
2015-10-06| 1343.0|
2013-06-13| 3399.0|
2014-02-22| 8472.0|
+----------+----------+
only showing top 20 rows

In [70]:
sales_sdf\
  .groupby("date", "shop_id")\
  .agg(
    F.sum(sales_sdf.item_cnt_day).alias("items_sold"),
    sales_sdf.item_cnt_day)\
  .orderBy("shop_id","date")\
  .show()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o4592.agg.
: org.apache.spark.sql.AnalysisException: expression '`item_cnt_day`' is neither present in the group by, nor is it an aggregate function. Add to group by or wrap in first() (or first_value) if you don't care which value you get.;;
Aggregate [date#1712, shop_id#1714], [date#1712, shop_id#1714, sum(cast(item_cnt_day#1717 as double)) AS items_sold#9005, item_cnt_day#1717]
+- Relation[date#1712,date_block_num#1713,shop_id#1714,item_id#1715,item_price#1716,item_cnt_day#1717] csv

	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.failAnalysis(CheckAnalysis.scala:46)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.failAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.org$apache$spark$sql$catalyst$analysis$CheckAnalysis$class$$anonfun$$checkValidAggregateExpression$1(CheckAnalysis.scala:235)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$10.apply(CheckAnalysis.scala:268)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1$$anonfun$apply$10.apply(CheckAnalysis.scala:268)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:268)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$$anonfun$checkAnalysis$1.apply(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:147)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis$class.checkAnalysis(CheckAnalysis.scala:89)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:103)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:117)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$$anonfun$executeAndCheck$1.apply(Analyzer.scala:114)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:205)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:114)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:85)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$analyzed$1.apply(QueryExecution.scala:86)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:111)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$executePhase$1.apply(QueryExecution.scala:229)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:835)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:228)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:83)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:83)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:75)
	at org.apache.spark.sql.Dataset$$anonfun$ofRows$1.apply(Dataset.scala:82)
	at org.apache.spark.sql.Dataset$$anonfun$ofRows$1.apply(Dataset.scala:80)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:835)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:80)
	at org.apache.spark.sql.RelationalGroupedDataset.toDF(RelationalGroupedDataset.scala:66)
	at org.a

In [71]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [72]:
matplotlib.rcParams['figure.figsize'] = [20, 10]
matplotlib.rcParams['font.size'] = 20

In [73]:
pdf = spark.sql("""
SELECT date_format(from_unixtime(unix_timestamp(date, 'dd.MM.yyyy')), "yyyy-MM-dd") AS date
,   SUM(item_cnt_day) AS items_sold
FROM sales
GROUP BY date
""").toPandas()
pdf.set_index("date")['items_sold'].plot()
plt.show()

In [74]:
import pandas as pd

In [75]:
#pdf.head()
#pdf['date'] = pd.to_datetime(pdf.date)
#pdf.date.dt.year.head()
pdf = pdf[pdf.date.dt.year==2014] #.head()
pdf.sort_values("date").set_index("date")['items_sold'].plot()
plt.show()

In [76]:
sales_sdf\
    .withColumn("date", F.unix_timestamp(F.col("date"), 'dd.MM.yyyy'))\
    .groupBy(F.col("date"))\
    .agg(F.sum(F.col("item_cnt_day")).alias("items_sold"))\
    .show()

__Exercise__ For each day what is total daily revenue? Output table should have columns `date`, `total_revenue`.

In [78]:
sales_sdf\
  .groupBy(F.col("date"))\
  .agg(F.sum(F.col("item_cnt_day") * F.col('item_price')).alias("total_revenue"))\
  .show()

+----------+------------------+
 date| total_revenue|
+----------+------------------+
16.02.2013| 4464531.999999987|
09.02.2014| 5148573.430000014|
01.09.2014| 2204969.260000013|
18.10.2014| 4852348.199999985|
27.06.2015|2387698.9800000004|
17.09.2015| 2522768.519999858|
29.04.2013| 1737224.999999997|
12.04.2013| 2497766.499999993|
18.09.2014| 2295228.0|
15.08.2015| 2045987.389999992|
28.10.2015|1722150.0999999908|
05.02.2013|2729662.2000001506|
21.09.2013| 5911155.490000767|
31.05.2014|5310957.9999999935|
02.11.2014| 3850449.230000004|
08.07.2015|2107555.7100000004|
13.09.2015|2637602.3900000174|
06.10.2015| 1339565.1|
13.06.2013| 1984354.390000003|
22.02.2014| 7644851.999999972|
+----------+------------------+
only showing top 20 rows

## Joins

In [80]:
sales_sdf.show()

+----------+--------------+-------+-------+----------+------------+
 date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
02.01.2013| 0| 59| 22154| 999.0| 1.0|
03.01.2013| 0| 25| 2552| 899.0| 1.0|
05.01.2013| 0| 25| 2552| 899.0| -1.0|
06.01.2013| 0| 25| 2554| 1709.05| 1.0|
15.01.2013| 0| 25| 2555| 1099.0| 1.0|
10.01.2013| 0| 25| 2564| 349.0| 1.0|
02.01.2013| 0| 25| 2565| 549.0| 1.0|
04.01.2013| 0| 25| 2572| 239.0| 1.0|
11.01.2013| 0| 25| 2572| 299.0| 1.0|
03.01.2013| 0| 25| 2573| 299.0| 3.0|
03.01.2013| 0| 25| 2574| 399.0| 2.0|
05.01.2013| 0| 25| 2574| 399.0| 1.0|
07.01.2013| 0| 25| 2574| 399.0| 1.0|
08.01.2013| 0| 25| 2574| 399.0| 2.0|
10.01.2013| 0| 25| 2574| 399.0| 1.0|
11.01.2013| 0| 25| 2574| 399.0| 2.0|
13.01.2013| 0| 25| 2574| 399.0| 1.0|
16.01.2013| 0| 25| 2574| 399.0| 1.0|
26.01.2013| 0| 25| 2574| 399.0| 1.0|
27.01.2013| 0| 25| 2574| 399.0| 1.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 20 rows

In [81]:
shops_sdf = spark.read.option("header", "true").csv("/FileStore/tables/shops.csv")

In [82]:
shops_sdf.show(truncate=False)

+----------------------------------+-------+
shop_name |shop_id|
+----------------------------------+-------+
!Якутск Орджоникидзе, 56 фран |0 |
"!Якутск ТЦ ""Центральный"" фран" |1 |
"Адыгея ТЦ ""Мега""" |2 |
"Балашиха ТРК ""Октябрь-Киномир"""|3 |
"Волжский ТЦ ""Волга Молл""" |4 |
"Вологда ТРЦ ""Мармелад""" |5 |
Воронеж (Плехановская, 13) |6 |
"Воронеж ТРЦ ""Максимир""" |7 |
"Воронеж ТРЦ Сити-Парк ""Град""" |8 |
Выездная Торговля |9 |
Жуковский ул. Чкалова 39м? |10 |
Жуковский ул. Чкалова 39м² |11 |
Интернет-магазин ЧС |12 |
"Казань ТЦ ""Бехетле""" |13 |
"Казань ТЦ ""ПаркХаус"" II" |14 |
"Калуга ТРЦ ""XXI век""" |15 |
"Коломна ТЦ ""Рио""" |16 |
"Красноярск ТЦ ""Взлетка Плаза""" |17 |
"Красноярск ТЦ ""Июнь""" |18 |
"Курск ТЦ ""Пушкинский""" |19 |
+----------------------------------+-------+
only showing top 20 rows

In [83]:
shops_sdf.createOrReplaceTempView("shops")

In [84]:
spark.sql("""
SELECT sales.shop_id
,   shops.shop_name
FROM (
    SELECT DISTINCT shop_id
    FROM sales)  sales
LEFT JOIN shops
ON sales.shop_id == shops.shop_id
""").show(truncate=False)

In [85]:
spark.sql("""
select
  shop_id,
  count(*) as nb
from shops
group by shop_id
having nb > 1
order by nb desc
limit 10
"""
).show()

+-------+---+
shop_id| nb|
+-------+---+
+-------+---+

In [86]:
spark.sql("""
SELECT 
    sales.shop_id
,   shops.shop_name
,   sum(sales.item_cnt_day)
FROM sales
LEFT JOIN shops
ON sales.shop_id = shops.shop_id
GROUP BY sales.shop_id, shops.shop_name
""").show(truncate=False)

In [87]:
spark.sql("""
SELECT 
    sales.shop_id
,   shops.shop_name
,   sum(sales.item_cnt_day)
FROM sales
LEFT JOIN shops
USING (shop_id)
GROUP BY sales.shop_id, shops.shop_name
""").show(truncate=False)

+-------+-----------------------------------------+---------------------------------+
shop_id|shop_name |sum(CAST(item_cnt_day AS DOUBLE))|
+-------+-----------------------------------------+---------------------------------+
52 |"Уфа ТК ""Центральный""" |49744.0 |
23 |"Москва ТК ""Буденовский"" (пав.А2)" |7705.0 |
49 |"Тюмень ТРЦ ""Кристалл""" |17090.0 |
29 |"Москва ТЦ ""Новый век"" (Новокосино)" |58713.0 |
2 |"Адыгея ТЦ ""Мега""" |30620.0 |
9 |Выездная Торговля |15866.0 |
39 |"РостовНаДону ТРК ""Мегацентр Горизонт"""|16617.0 |
6 |Воронеж (Плехановская, 13) |100489.0 |
5 |"Вологда ТРЦ ""Мармелад""" |42762.0 |
14 |"Казань ТЦ ""ПаркХаус"" II" |46375.0 |
1 |"!Якутск ТЦ ""Центральный"" фран" |6311.0 |
19 |"Курск ТЦ ""Пушкинский""" |73455.0 |
13 |"Казань ТЦ ""Бехетле""" |19763.0 |
51 |"Тюмень ТЦ ""Зеленый Берег""" |48767.0 |
4 |"Волжский ТЦ ""Волга Молл""" |43942.0 |
16 |"Коломна ТЦ ""Рио""" |61633.0 |
53 |"Уфа ТЦ ""Семья"" 2" |61657.0 |
38 |"Омск ТЦ ""Мега""" |53886.0 |
11 |Жуковский ул. Чкалова 39м² |572.0 |
37 |"Новосибирск ТЦ ""Мега""" |46256.0 |
+-------+-----------------------------------------+---------------------------------+
only showing top 20 rows

In [88]:
sales_sdf\
  .join(shops_sdf, "shop_id", how="left")\
  .groupBy("shop_id", "shop_name")\
  .agg(F.sum(sales_sdf.item_cnt_day))\
  .show()

+-------+--------------------+-----------------+
shop_id| shop_name|sum(item_cnt_day)|
+-------+--------------------+-----------------+
 52|"Уфа ТК ""Централ...| 49744.0|
 23|"Москва ТК ""Буде...| 7705.0|
 49|"Тюмень ТРЦ ""Кри...| 17090.0|
 29|"Москва ТЦ ""Новы...| 58713.0|
 2|"Адыгея ТЦ ""Мега"""| 30620.0|
 9| Выездная Торговля| 15866.0|
 39|"РостовНаДону ТРК...| 16617.0|
 6|Воронеж (Плеханов...| 100489.0|
 5|"Вологда ТРЦ ""Ма...| 42762.0|
 14|"Казань ТЦ ""Парк...| 46375.0|
 1|"!Якутск ТЦ ""Цен...| 6311.0|
 19|"Курск ТЦ ""Пушки...| 73455.0|
 13|"Казань ТЦ ""Бехе...| 19763.0|
 51|"Тюмень ТЦ ""Зеле...| 48767.0|
 4|"Волжский ТЦ ""Во...| 43942.0|
 16|"Коломна ТЦ ""Рио"""| 61633.0|
 53|"Уфа ТЦ ""Семья"" 2"| 61657.0|
 38| "Омск ТЦ ""Мега"""| 53886.0|
 11|Жуковский ул. Чка...| 572.0|
 37|"Новосибирск ТЦ "...| 46256.0|
+-------+--------------------+-----------------+
only showing top 20 rows

In [89]:
sales_sdf\
    .select("shop_id")\
    .distinct()\
    .join(shops_sdf, "shop_id", how="left")\
    .select("shop_id", "shop_name")\
    .show(truncate=False)

__Exercise__ Find a product with highest sell.

In [91]:
# example with cache
# example using struct data
# example using subqueries (explicit or as two dataframes)

In [92]:
item_daily_sales = sales_sdf\
  .groupBy("date","item_id")\
  .agg(F.sum(F.col("item_cnt_day")).alias("item_cnt_day")).cache()
item_daily_sales.show()

+----------+-------+------------+
 date|item_id|item_cnt_day|
+----------+-------+------------+
12.01.2013| 2715| 1.0|
18.01.2013| 2257| 2.0|
27.01.2013| 3175| 1.0|
25.01.2013| 3158| 1.0|
26.01.2013| 1333| 1.0|
07.01.2013| 1832| 1.0|
18.01.2013| 5459| 2.0|
06.01.2013| 5574| 11.0|
05.01.2013| 6857| 2.0|
28.01.2013| 14827| 4.0|
25.01.2013| 12555| 1.0|
07.01.2013| 13158| 4.0|
16.01.2013| 12676| 1.0|
18.01.2013| 12654| 1.0|
27.01.2013| 12967| 1.0|
08.01.2013| 16656| 1.0|
20.01.2013| 16832| 5.0|
10.01.2013| 15258| 1.0|
12.01.2013| 15558| 2.0|
09.01.2013| 15838| 1.0|
+----------+-------+------------+
only showing top 20 rows

In [93]:
max_daily_sale = item_daily_sales\
  .groupBy("date")\
  .agg(F.max("item_cnt_day").alias("max_item_cnt_day"))
max_daily_sale.show()

+----------+----------------+
 date|max_item_cnt_day|
+----------+----------------+
16.02.2013| 118.0|
09.02.2014| 292.0|
01.09.2014| 205.0|
18.10.2014| 340.0|
27.06.2015| 153.0|
17.09.2015| 153.0|
12.04.2013| 147.0|
29.04.2013| 68.0|
18.09.2014| 174.0|
15.08.2015| 112.0|
28.10.2015| 2169.0|
05.02.2013| 390.0|
21.09.2013| 541.0|
31.05.2014| 307.0|
02.11.2014| 308.0|
08.07.2015| 91.0|
13.09.2015| 140.0|
06.10.2015| 131.0|
13.06.2013| 211.0|
22.02.2014| 506.0|
+----------+----------------+
only showing top 20 rows

In [94]:
max_daily_sale.alias("ms")\
  .join(item_daily_sales.alias("ds"), F.expr("(ms.date = ds.date) AND (ms.max_item_cnt_day = ds.item_cnt_day)"))\
  .select("ds.date","item_id","item_cnt_day")\
  .orderBy("date")\
  .show()

+----------+-------+------------+
 date|item_id|item_cnt_day|
+----------+-------+------------+
01.01.2013| 16450| 39.0|
01.01.2014| 20949| 139.0|
01.01.2015| 20949| 168.0|
01.02.2013| 19811| 62.0|
01.02.2014| 20949| 365.0|
01.02.2015| 20949| 133.0|
01.03.2013| 2445| 56.0|
01.03.2014| 20949| 367.0|
01.03.2015| 20949| 90.0|
01.04.2013| 1939| 61.0|
01.04.2014| 20949| 111.0|
01.04.2015| 20949| 31.0|
01.05.2013| 20949| 115.0|
01.05.2014| 20949| 248.0|
01.05.2015| 20949| 168.0|
01.06.2013| 20949| 397.0|
01.06.2014| 20949| 252.0|
01.06.2015| 20949| 92.0|
01.07.2013| 20949| 224.0|
01.07.2014| 20949| 189.0|
+----------+-------+------------+
only showing top 20 rows

In [95]:
item_and_sales = item_daily_sales\
  .select("date", F.struct(F.col("item_cnt_day"),F.col("item_id")).alias("par"))\
  .groupBy("date")\
  .agg(F.max("par").alias("best_sale"))\
  .select("date","best_sale.item_cnt_day","best_sale.item_id")\
  .orderBy("date")
item_and_sales.show()

+----------+------------+-------+
 date|item_cnt_day|item_id|
+----------+------------+-------+
01.01.2013| 39.0| 16450|
01.01.2014| 139.0| 20949|
01.01.2015| 168.0| 20949|
01.02.2013| 62.0| 19811|
01.02.2014| 365.0| 20949|
01.02.2015| 133.0| 20949|
01.03.2013| 56.0| 2445|
01.03.2014| 367.0| 20949|
01.03.2015| 90.0| 20949|
01.04.2013| 61.0| 1939|
01.04.2014| 111.0| 20949|
01.04.2015| 31.0| 20949|
01.05.2013| 115.0| 20949|
01.05.2014| 248.0| 20949|
01.05.2015| 168.0| 20949|
01.06.2013| 397.0| 20949|
01.06.2014| 252.0| 20949|
01.06.2015| 92.0| 20949|
01.07.2013| 224.0| 20949|
01.07.2014| 189.0| 20949|
+----------+------------+-------+
only showing top 20 rows

In [96]:
item_and_sales.printSchema()

root
-- date: string (nullable = true)
-- best_sale: struct (nullable = true)
 |-- item_cnt_day: double (nullable = true)
 |-- item_id: string (nullable = true)

Bonus: anadir el nombre del producto ? (indice `items.csv`)

__Exercise__ For each day in August 2015 find shop name with maximal sales.

## Window functions

First let's correct the date.

In [101]:
item_daily_sales.createOrReplaceTempView("item_daily_sales")

In [102]:
item_daily_sales.show()

+----------+-------+------------+
 date|item_id|item_cnt_day|
+----------+-------+------------+
12.01.2013| 2715| 1.0|
18.01.2013| 2257| 2.0|
27.01.2013| 3175| 1.0|
25.01.2013| 3158| 1.0|
26.01.2013| 1333| 1.0|
07.01.2013| 1832| 1.0|
18.01.2013| 5459| 2.0|
06.01.2013| 5574| 11.0|
05.01.2013| 6857| 2.0|
28.01.2013| 14827| 4.0|
25.01.2013| 12555| 1.0|
07.01.2013| 13158| 4.0|
16.01.2013| 12676| 1.0|
18.01.2013| 12654| 1.0|
27.01.2013| 12967| 1.0|
08.01.2013| 16656| 1.0|
20.01.2013| 16832| 5.0|
10.01.2013| 15258| 1.0|
12.01.2013| 15558| 2.0|
09.01.2013| 15838| 1.0|
+----------+-------+------------+
only showing top 20 rows

In [103]:
spark.sql("""
SELECT
LEAD(Struct(date, item_id)) OVER (partition by date order by item_cnt_day desc) as item_id
FROM sales
order by date
""").show()

+-------------------+
 item_id|
+-------------------+
[01.01.2013, 19637]|
 [01.01.2013, 9043]|
[01.01.2013, 22088]|
 [01.01.2013, 98]|
 [01.01.2013, 3223]|
[01.01.2013, 16428]|
[01.01.2013, 15100]|
[01.01.2013, 16450]|
 [01.01.2013, 1471]|
 [01.01.2013, 3321]|
[01.01.2013, 22092]|
[01.01.2013, 22092]|
[01.01.2013, 16451]|
 [01.01.2013, 3432]|
[01.01.2013, 13351]|
 [01.01.2013, 3432]|
[01.01.2013, 16450]|
[01.01.2013, 11041]|
 [01.01.2013, 1470]|
 [01.01.2013, 1470]|
+-------------------+
only showing top 20 rows

In [104]:
spark.sql("""
SELECT *
,   date_format(from_unixtime(unix_timestamp(date, 'dd.MM.yyyy')), "yyyy-MM-dd") AS date_iso
FROM sales
""").createOrReplaceTempView("sales_iso")

In [105]:
sales_iso_sdf = sales_sdf\
    .withColumn(
    "date_iso", 
    F.unix_timestamp(F.col("date"), 'dd.MM.yyyy'))
sales_iso_sdf.show()

In [106]:
spark.sql("""
SELECT shop_id
,   item_id
,   date_iso
,   item_cnt_day
,   LEAD(item_cnt_day) OVER 
        (PARTITION BY shop_id, item_id ORDER BY date_iso) as lead_item_cnt_day
FROM sales_iso
ORDER BY shop_id
,   item_id
,   date_iso
""").show()

In [107]:
from pyspark.sql import Window

sales_iso_sdf.select(
    F.col("shop_id"),   
    F.col("item_id"),   
    F.col("date_iso"),   
    F.col("item_cnt_day"),   
    F.lead(F.col("item_cnt_day"))\
        .over(Window.partitionBy("shop_id", "item_id").orderBy('date_iso'))\
        .alias("lead_item_cnt_day")
).orderBy("shop_id", "item_id", "date_iso").show()


In [108]:
window = Window.partitionBy("shop_id", "item_id").orderBy('date_iso')

sales_iso_sdf.select(
    F.col("shop_id"),   
    F.col("item_id"),   
    F.col("date_iso"),   
    F.col("item_cnt_day"),   
    F.lead(F.col("item_cnt_day"))\
        .over(window)\
        .alias("lead_item_cnt_day")
).orderBy("shop_id", "item_id", "date_iso").show()


__Exercise__ What is moving average (+-3 days) of total daily revenue?

## Extra Exercises

* Which shop had the highest sells in August 2015. What was his name?
* What is the name of category of with the highest monthly sells.

# DataFrame as an RDD of Rows

In [113]:
type(sales_sdf)

In [114]:
row = sales_sdf.first()
row

In [115]:
print(row.date)
print(row['date_block_num'])
row.asDict()

In [116]:
sales_sdf.rdd.take(5)

In [117]:
print(sales_sdf.rdd.toDebugString().decode())

In [118]:
sales_per_day = sales_sdf\
    .groupBy(F.col("date"))\
    .agg(F.sum(F.col("item_cnt_day")))

print(sales_per_day.rdd.toDebugString().decode())